# lab 6 - servers simulation
- Ricardo Méndez 21289
- Sara Echeverría 21371
- Melissa Pérez 21385

Suponga que usted está a cargo de definir la arquitectura para el lanzamiento de su próxima aplicación web, C3 (sistema de contabilidad de la carreta contadora). La junta directiva le ha solicitado encontrar el mejor servicio de hosting para el proyecto. Tras una extensa investigación, concluye que las mejores opciones son: 
- Proveedor 1 - Mountain Mega Computing, con una infraestructura de servidor único que puede atender hasta 100 solicitudes por segundo.
- Proveedor 2 - Pizzita Computing, con una infraestructura en la nube de múltiples servidores medianamente potentes, donde cada servidor tiene una décima parte de la potencia del servidor de Mountain Mega Computing, y se paga solo por los servidores necesarios. 

Las pruebas de estrés y las proyecciones indican que su aplicación no excederá las 2,400 solicitudes por minuto durante los primeros dos años. Un análisis de sistemas similares sugiere que las solicitudes seguirán un proceso de Poisson, con tiempos de servicio modelados por una variable exponencial. Como debe presentar su decisión mañana, opta por realizar una simulación basada en la promoción de los proveedores para concluir cuál será la mejor opción.

In [1]:
import numpy as np
from scipy.stats import expon

In [2]:
def simulateProvider1(lambdaRate, serviceRate, simulationTime):
    totalSimulationSeconds = simulationTime * 3600
    interArrivalTimes = expon.rvs(scale=1/lambdaRate, size=int(lambdaRate * totalSimulationSeconds))
    serviceTimes = expon.rvs(scale=1/serviceRate, size=len(interArrivalTimes))
    
    # Métricas
    requestsHandled = 0
    totalWaitTime = 0
    totalServiceTime = 0
    idleTime = 0
    lastDeparture = 0
    
    queue = []
    currentTime = 0
    serverBusy = False
    serviceStartTime = 0
    
    for i in range(len(interArrivalTimes)):
        currentTime += interArrivalTimes[i]
        
        if serverBusy:
            if currentTime >= serviceStartTime + serviceTimes[i]:
                serverBusy = False
                lastDeparture = serviceStartTime + serviceTimes[i]
                totalServiceTime += serviceTimes[i]
                requestsHandled += 1
            else:
                queue.append(serviceTimes[i])
        if not serverBusy and len(queue) == 0:
            serverBusy = True
            serviceStartTime = currentTime
        elif len(queue) > 0:
            # Atender la siguiente solicitud en la cola
            nextServiceTime = queue.pop(0)
            totalWaitTime += (currentTime - serviceStartTime)
            serverBusy = True
            serviceStartTime = currentTime
    
    idleTime = totalSimulationSeconds - totalServiceTime
    
    averageWaitTime = totalWaitTime / requestsHandled if requestsHandled > 0 else 0
    utilization = totalServiceTime / totalSimulationSeconds
    avgQueueLength = totalWaitTime / totalSimulationSeconds

    return requestsHandled, totalServiceTime, idleTime, totalWaitTime, averageWaitTime, avgQueueLength, lastDeparture

In [3]:
def simulateProvider2(lambdaRate, serverPower, numServers, simulationTime):
    totalSimulationSeconds = simulationTime * 3600
    interArrivalTimes = expon.rvs(scale=1/lambdaRate, size=int(lambdaRate * totalSimulationSeconds))
    serviceTimes = expon.rvs(scale=1/serverPower, size=len(interArrivalTimes))
    
    # Métricas
    requestsHandled = 0
    totalWaitTime = 0
    totalServiceTime = [0] * numServers  # Tiempo ocupado por cada servidor
    idleTime = 0
    lastDeparture = 0
    
    serverBusyUntil = [0] * numServers  # Tiempo hasta el que cada servidor está ocupado
    queue = []
    currentTime = 0
    
    for i in range(len(interArrivalTimes)):
        currentTime += interArrivalTimes[i]
        
        # Revisamos si hay algún servidor disponible
        serverAssigned = False
        for j in range(numServers):
            if serverBusyUntil[j] <= currentTime:
                if len(queue) > 0:
                    # Atender la siguiente solicitud en la cola
                    nextServiceTime = queue.pop(0)
                    totalWaitTime += (currentTime - serverBusyUntil[j])
                    serverBusyUntil[j] = currentTime + nextServiceTime
                    totalServiceTime[j] += nextServiceTime
                    requestsHandled += 1
                    serverAssigned = True
                    lastDeparture = max(lastDeparture, serverBusyUntil[j])
                elif len(queue) == 0:
                    # Si no hay cola, atender esta solicitud
                    serverBusyUntil[j] = currentTime + serviceTimes[i]
                    totalServiceTime[j] += serviceTimes[i]
                    requestsHandled += 1
                    serverAssigned = True
                    lastDeparture = max(lastDeparture, serverBusyUntil[j])
                break
        
        if not serverAssigned:
            # Si todos los servidores están ocupados, añadir a la cola
            queue.append(serviceTimes[i])
    
    # Calcular el tiempo total ocupado y desocupado
    totalServiceTimeSum = sum(totalServiceTime)
    idleTime = totalSimulationSeconds * numServers - totalServiceTimeSum
    
    averageWaitTime = totalWaitTime / requestsHandled if requestsHandled > 0 else 0
    utilization = totalServiceTimeSum / (totalSimulationSeconds * numServers)
    avgQueueLength = totalWaitTime / totalSimulationSeconds

    return requestsHandled, totalServiceTimeSum, idleTime, totalWaitTime, averageWaitTime, avgQueueLength, lastDeparture

In [4]:
def runSimulations(lambdaRate, provider1ServiceRate, provider2ServerPower, provider2NumServers, simulationTime):
    # simulación para el Proveedor 1
    requestsHandled1, totalServiceTime1, idleTime1, totalWaitTime1, averageWaitTime1, avgQueueLength1, lastDeparture1 = simulateProvider1(lambdaRate, provider1ServiceRate, simulationTime)
    
    # simulación para el Proveedor 2
    requestsHandled2, totalServiceTime2, idleTime2, totalWaitTime2, averageWaitTime2, avgQueueLength2, lastDeparture2 = simulateProvider2(lambdaRate, provider2ServerPower, provider2NumServers, simulationTime)
    
    # resultados para el Proveedor 1
    print("Proveedor 1 (Mountain Mega Computing):")
    print(f"- Solicitudes atendidas: {requestsHandled1}")
    print(f"- Tiempo ocupado: {totalServiceTime1:.2f} segundos")
    print(f"- Tiempo desocupado (idle): {idleTime1:.2f} segundos")
    print(f"- Tiempo total en cola: {totalWaitTime1:.2f} segundos")
    print(f"- Tiempo promedio en cola: {averageWaitTime1:.4f} segundos")
    print(f"- Promedio de solicitudes en cola por segundo: {avgQueueLength1:.4f}")
    print(f"- Momento de la salida de la última solicitud: {lastDeparture1:.2f} segundos\n")
    
    # resultados para el Proveedor 2
    print("Proveedor 2 (Pizzita Computing):")
    print(f"- Solicitudes atendidas: {requestsHandled2}")
    print(f"- Tiempo ocupado: {totalServiceTime2:.2f} segundos")
    print(f"- Tiempo desocupado (idle): {idleTime2:.2f} segundos")
    print(f"- Tiempo total en cola: {totalWaitTime2:.2f} segundos")
    print(f"- Tiempo promedio en cola: {averageWaitTime2:.4f} segundos")
    print(f"- Promedio de solicitudes en cola por segundo: {avgQueueLength2:.4f}")
    print(f"- Momento de la salida de la última solicitud: {lastDeparture2:.2f} segundos")

In [5]:
# Parámetros de la simulación
lambdaRate = 40                     # Tasa de solicitudes por segundo (2,400 solicitudes por minuto)
provider1ServiceRate = 100          # Capacidad de servicio del Proveedor 1 (Mountain Mega Computing)
provider2ServerPower = 10           # Capacidad de servicio de un servidor de Pizzita Computing
provider2NumServers = 5             # Número de servidores para Pizzita Computing
simulationTime = 1                  # Tiempo de simulación en horas

# Ejecutar la simulación para ambos proveedores
runSimulations(lambdaRate, provider1ServiceRate, provider2ServerPower, provider2NumServers, simulationTime)

Proveedor 1 (Mountain Mega Computing):
- Solicitudes atendidas: 102928
- Tiempo ocupado: 736.94 segundos
- Tiempo desocupado (idle): 2863.06 segundos
- Tiempo total en cola: 294.05 segundos
- Tiempo promedio en cola: 0.0029 segundos
- Promedio de solicitudes en cola por segundo: 0.0817
- Momento de la salida de la última solicitud: 3598.27 segundos

Proveedor 2 (Pizzita Computing):
- Solicitudes atendidas: 115103
- Tiempo ocupado: 11508.13 segundos
- Tiempo desocupado (idle): 6491.87 segundos
- Tiempo total en cola: 799.42 segundos
- Tiempo promedio en cola: 0.0069 segundos
- Promedio de solicitudes en cola por segundo: 0.2221
- Momento de la salida de la última solicitud: 3582.08 segundos


## Tasks
1. Modele, simule y analice el comportamiento de ambos sistemas durante una hora de ejecución de C3, y para cada sistema responda
- ¿Cuántas solicitudes atendió cada servidor?
    - Mountain Mega Computing: 102,928 solicitudes
    - Pizzita Computing: 115,103 solicitudes
- ¿Cuánto tiempo estuvo cada servidor ocupado?
    - Mountain Mega Computing: 736.94 segundos de 3,600 segundos.
    - Pizzita Computing: 11,508.13 segundos de 3,600 segundos. Lo cual parece extraño, pero es por el uso de múltiples servidores.
- ¿Cuánto tiempo estuvo cada servidor desocupado (iddle)?
    - Mountain Mega Computing: 2,863.06 segundos
    - Pizzita Computing: 6,491.87 segundos
- ¿Cuánto tiempo en total estuvieron las solicitudes en cola?
    - Mountain Mega Computing: 294.05 segundos
    - Pizzita Computing: 799.42 segundos
- En promedio ¿cuánto tiempo estuvo cada solicitud en cola?
    - Mountain Mega Computing: 0.0029 segundos
    - Pizzita Computing: 0.0069 segundos
- En promedio, ¿cuántas solicitudes estuvieron en cola cada segundo?
    - Mountain Mega Computing: 0.0817 solicitudes en cola por segundo
    - Pizzita Computing: 0.2221 solicitudes en cola por segundo
- ¿Cuál es el momento de la salida de la última solicitud?
    - Mountain Mega Computing: 3,598.27
    - Pizzita Computing: 3,582.08

2. Determine empíricamente cuántos servidores se necesitaría “alquilar” en Pizzita computing para asegurar que siempre habrá al menos un servidor disponible para atender una solicitud dada (en otras palabras, una solicitud nunca tiene que esperar en cola)

3. Se espera que a partir del tercer año del lanzamiento de su aplicación, la cantidad de usuarios sufra un alza, y por tanto deberán atender como máximo 6000 solicitudes por minuto. Resuelva el inciso 1 y 2 para esta nueva configuración.

4. Emita una recomendación para la junta directiva